In [1]:
from utils import *
from sympy import symbols, trigsimp, lambdify, diff, init_printing
init_printing(use_unicode=True)

In [2]:
# Joint parameters declaration
q1, q2, d3, q4, q5, q6 = symbols("q1 q2 d3 q4 q5 q6", real=True)

In [3]:
# Link length initialization
l1, l2, l3 = 10, 20, 30

In [4]:
# Forward Kinematics
H = Rz(q1)*Tz(l1)*Tx(-l2)*Rx(q2)*Tz(l3)*Tz(d3)*Rz(q4)*Rx(q5)*Rz(q6)
H = trigsimp(H)
H

⎡(-(sin(q₁)⋅cos(q₂)⋅cos(q₄) + sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₂)⋅sin(q
⎢                                                                             
⎢((-sin(q₁)⋅sin(q₄) + cos(q₁)⋅cos(q₂)⋅cos(q₄))⋅cos(q₅) - sin(q₂)⋅sin(q₅)⋅cos(q
⎢                                                                             
⎢                                 (sin(q₂)⋅cos(q₄)⋅cos(q₅) + sin(q₅)⋅cos(q₂))⋅
⎢                                                                             
⎣                                                                       0     

₅))⋅sin(q₆) + (-sin(q₁)⋅sin(q₄)⋅cos(q₂) + cos(q₁)⋅cos(q₄))⋅cos(q₆)  (-(sin(q₁)
                                                                              
₁))⋅sin(q₆) + (sin(q₁)⋅cos(q₄) + sin(q₄)⋅cos(q₁)⋅cos(q₂))⋅cos(q₆)   ((-sin(q₁)
                                                                              
sin(q₆) + sin(q₂)⋅sin(q₄)⋅cos(q₆)                                             
                                                   

In [5]:
# General Rotation Matrix
R = H[0:3, :-1]
R

⎡(-(sin(q₁)⋅cos(q₂)⋅cos(q₄) + sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₂)⋅sin(q
⎢                                                                             
⎢((-sin(q₁)⋅sin(q₄) + cos(q₁)⋅cos(q₂)⋅cos(q₄))⋅cos(q₅) - sin(q₂)⋅sin(q₅)⋅cos(q
⎢                                                                             
⎣                                 (sin(q₂)⋅cos(q₄)⋅cos(q₅) + sin(q₅)⋅cos(q₂))⋅

₅))⋅sin(q₆) + (-sin(q₁)⋅sin(q₄)⋅cos(q₂) + cos(q₁)⋅cos(q₄))⋅cos(q₆)  (-(sin(q₁)
                                                                              
₁))⋅sin(q₆) + (sin(q₁)⋅cos(q₄) + sin(q₄)⋅cos(q₁)⋅cos(q₂))⋅cos(q₆)   ((-sin(q₁)
                                                                              
sin(q₆) + sin(q₂)⋅sin(q₄)⋅cos(q₆)                                             

⋅cos(q₂)⋅cos(q₄) + sin(q₄)⋅cos(q₁))⋅cos(q₅) + sin(q₁)⋅sin(q₂)⋅sin(q₅))⋅cos(q₆)
                                                                              
⋅sin(q₄) + cos(q₁)⋅cos(q₂)⋅cos(q₄))⋅cos(q₅) - sin(

In [6]:
# General Translational Matrix
T = H[0:3, -1]
T

⎡d₃⋅sin(q₁)⋅sin(q₂) + 30⋅sin(q₁)⋅sin(q₂) - 20⋅cos(q₁) ⎤
⎢                                                     ⎥
⎢-d₃⋅sin(q₂)⋅cos(q₁) - 20⋅sin(q₁) - 30⋅sin(q₂)⋅cos(q₁)⎥
⎢                                                     ⎥
⎣            d₃⋅cos(q₂) + 30⋅cos(q₂) + 10             ⎦

In [7]:
# Calculating Jacobian Matrix
# Running this cell takes about 10 minutes
# Maybe Jump by 2 cells, to load the Jacobian Function from File
Jaco = Matrix([])

var = [q1, q2, d3, q4, q5, q6]
for j in var:
    HD = trigsimp(diff(H, j))

    T1 = HD[0:3, -1]

    RD = HD[0:3, :-1]
    RW = trigsimp(RD*R.T)

    J = T1.row_insert(3, Matrix([RW[2,1], RW[0,2], RW[1,0]]))

    Jaco = Jaco.col_insert(len(Jaco), J)

Jaco

⎡d₃⋅sin(q₂)⋅cos(q₁) + 20⋅sin(q₁) + 30⋅sin(q₂)⋅cos(q₁)  (d₃ + 30)⋅sin(q₁)⋅cos(q
⎢                                                                             
⎢d₃⋅sin(q₁)⋅sin(q₂) + 30⋅sin(q₁)⋅sin(q₂) - 20⋅cos(q₁)  -(d₃ + 30)⋅cos(q₁)⋅cos(
⎢                                                                             
⎢                         0                                -(d₃ + 30)⋅sin(q₂) 
⎢                                                                             
⎢                         0                                     cos(q₁)       
⎢                                                                             
⎢                         0                                     sin(q₁)       
⎢                                                                             
⎣                         1                                        0          

₂)   sin(q₁)⋅sin(q₂)          0                              0                
                                                   

In [8]:
# Loading Jacobian Function to file to prevent having to run everytime
import dill
dill.settings["recurse"]=True

J_Function = lambdify([(q1, q2, d3, q4, q5, q6)], Jaco, "numpy")
dill.dump(J_Function, open("J_Function", "wb"))

In [9]:
J_Func = dill.load(open("J_Function", "rb"))
A = J_Func((0,0,0,0,0,0))
A

array([[  0.,   0.,   0.,   0.,   0.,   0.],
       [-20., -30.,  -0.,   0.,   0.,   0.],
       [  0.,  -0.,   1.,   0.,   0.,   0.],
       [  0.,   1.,   0.,   0.,   1.,   0.],
       [  0.,   0.,   0.,  -0.,   0.,   0.],
       [  1.,   0.,   0.,   1.,   0.,   1.]])